アンサンブル学習

In [8]:
# データセット準備
import pandas as pd
import numpy as np

train_data=pd.read_csv("/Users/chidayasuhiro/diveintocode-ml/train.csv")

# Pandasからndarrayへ変更

X =train_data.loc[:,['GrLivArea','YearBuilt']]
y =train_data.loc[:,"SalePrice"]
Xandy=pd.concat([X,y],axis=1)

X_np=np.array(X)
y_np=np.array(y)

In [18]:
#　分割と標準化
# 分割
from sklearn.model_selection import train_test_split
train_X,test_X,train_y,test_y=train_test_split(X_np,y_np)

from sklearn.preprocessing import StandardScaler

train_X1=train_X.astype(float)
test_X1=test_X.astype(float)
scaler = StandardScaler()
scaler.fit(train_X1)
train_X1sd=scaler.transform(train_X1)
test_X1sd=scaler.transform(test_X1)

【問題1】ブレンディングのスクラッチ実装

パターン１　線形回帰と決定木によるブレンディング

In [19]:
#　線形回帰による学習
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(train_X,train_y)
y_pred1=reg.predict(test_X)
from sklearn.metrics import mean_squared_error
mean_squared_error(test_y, y_pred1)

2780059954.010316

In [20]:
# 決定木による学習
from sklearn.tree import DecisionTreeRegressor
DTR = DecisionTreeRegressor(random_state=0)
DTR.fit(train_X,train_y)
y_pred2=DTR.predict(test_X)
from sklearn.metrics import mean_squared_error
mean_squared_error(test_y, y_pred2)

4508115881.435921

In [12]:
# 個々の推定結果を与えるとそれらの平均の推定結果を返す関数

def pred_func(pred_list):
    """
    pred_list : nadarray(n_sample,sample_function)
    
    """
    y_pred=[]
    for i in range(len(pred_list)):
        pred=sum(pred_list[i])/len(pred_list[1])
        y_pred.append(pred)
    return y_pred

In [21]:
# 実装例
# 線形回帰と決定木の結果
pred_list1=np.vstack((y_pred1,y_pred2)).T
# 制作した関数に入れ平均を得る
blending_pred=pred_func(pred_list1)

In [22]:
#　平均二乗誤差を計算し精度を確認

from sklearn.metrics import mean_squared_error
mean_squared_error(test_y, blending_pred)

3122605268.3684034

●１つ目の例として、単一の決定木より精度が上がったことがわかった。

パターン２　SVMのカーネルの違いによるブレンディング

In [24]:
# SVMの「kernel='rbf」による学習

from sklearn.svm import SVR
svm=SVR()
svm.fit(train_X,train_y)
y_pred3=svm.predict(test_X)
from sklearn.metrics import mean_squared_error
mean_squared_error(test_y, y_pred3)

6321589994.02816

In [25]:
# SVMの「kernel='linear'」による学習
svm1=SVR(kernel="linear")
svm1.fit(train_X,train_y)
y_pred4=svm1.predict(test_X)
from sklearn.metrics import mean_squared_error
mean_squared_error(test_y, y_pred4)

2669972880.690646

In [26]:
# SVMの「kernel="sigmoid"」による学習
svm2=SVR(kernel="sigmoid")
svm2.fit(train_X,train_y)
y_pred5=svm2.predict(test_X)
from sklearn.metrics import mean_squared_error
mean_squared_error(test_y, y_pred5)

6321611846.906849

In [39]:
# 実装
# 制作した関数に入れ平均を得る
pred_list2=np.vstack((y_pred3,y_pred4,y_pred5)).T
blending_pred2=pred_func(pred_list2)
#　平均二乗誤差を計算し精度を確認
from sklearn.metrics import mean_squared_error
mean_squared_error(test_y, blending_pred2)

4350145758.036012

●２つ目の例として、単一の「kernel='rbf'」と「kernel="sigmoid"」より精度が上がったことがわかった。

パターン３　対数変換の違いによるブレンディング

In [32]:
#　通常の変数のままの線形回帰による学習
from sklearn.linear_model import LinearRegression
reg1 = LinearRegression()
reg1.fit(train_X,train_y)
y_pred6=reg1.predict(test_X)
from sklearn.metrics import mean_squared_error
mean_squared_error(test_y, y_pred6)

2780059954.010316

In [37]:
#　対数変換した変数の線形回帰による学習
from sklearn.linear_model import LinearRegression
reg2 = LinearRegression()
reg2.fit(train_X1log,train_y)
y_pred7=reg2.predict(test_X1log)
from sklearn.metrics import mean_squared_error
mean_squared_error(test_y, y_pred7)

2462264344.100039

In [38]:
# 実装
# 制作した関数に入れ平均を得る
pred_list3=np.vstack((y_pred6,y_pred7)).T
blending_pred3=pred_func(pred_list3)
#　平均二乗誤差を計算し精度を確認
from sklearn.metrics import mean_squared_error
mean_squared_error(test_y, blending_pred3)

2544257059.257657

●３つ目の例として、対数化することで単一の線形回帰より精度が上がったことがわかった。

【問題2】バギングのスクラッチ実装

線形回帰により５種類のサブセットでN個学習し推定結果の平均を取ってみる

In [44]:
# 1
from sklearn.model_selection import train_test_split
train_X1,test_X1,train_y1,test_y1=train_test_split(X_np,y_np)
from sklearn.linear_model import LinearRegression
reg11 = LinearRegression()
reg11.fit(train_X1,train_y1)
y_pred11=reg11.predict(test_X1)
from sklearn.metrics import mean_squared_error
print(mean_squared_error(test_y, y_pred11))

# 2
from sklearn.model_selection import train_test_split
train_X2,test_X2,train_y2,test_y2=train_test_split(X_np,y_np)
from sklearn.linear_model import LinearRegression
reg12 = LinearRegression()
reg12.fit(train_X2,train_y2)
y_pred12=reg12.predict(test_X2)
from sklearn.metrics import mean_squared_error
print(mean_squared_error(test_y, y_pred12))

# 3
from sklearn.model_selection import train_test_split
train_X3,test_X3,train_y3,test_y3=train_test_split(X_np,y_np)
from sklearn.linear_model import LinearRegression
reg13 = LinearRegression()
reg13.fit(train_X3,train_y3)
y_pred13=reg13.predict(test_X3)
from sklearn.metrics import mean_squared_error
print(mean_squared_error(test_y, y_pred13))

# 4
from sklearn.model_selection import train_test_split
train_X4,test_X4,train_y4,test_y4=train_test_split(X_np,y_np)
from sklearn.linear_model import LinearRegression
reg14 = LinearRegression()
reg14.fit(train_X4,train_y4)
y_pred14=reg14.predict(test_X4)
from sklearn.metrics import mean_squared_error
print(mean_squared_error(test_y, y_pred14))

# 5
from sklearn.model_selection import train_test_split
train_X5,test_X5,train_y5,test_y5=train_test_split(X_np,y_np)
from sklearn.linear_model import LinearRegression
reg15 = LinearRegression()
reg15.fit(train_X5,train_y5)
y_pred15=reg15.predict(test_X5)
from sklearn.metrics import mean_squared_error
print(mean_squared_error(test_y, y_pred15))

10944360174.061834
10530922698.621288
9537159080.469467
9804105429.485977
9150530583.735205


In [45]:
# 制作した関数に入れ平均を得る
pred_list4=np.vstack((y_pred11,y_pred12,y_pred13,y_pred14,y_pred15)).T
blending_pred4=pred_func(pred_list4)
#　平均二乗誤差を計算し精度を確認
from sklearn.metrics import mean_squared_error
mean_squared_error(test_y, blending_pred4)

6740126327.323547

５回分で行ったところ、平均二乗誤差はかなり小さくなり精度が上がったことがわかった

【問題3】スタッキングのスクラッチ実装

分割の数Kは３、モデルの数Mは２で行う

モデルの１つは線形回帰、もう一つは決定木で行う。

第二段階で行うモデルは線形回帰を使用する

In [46]:
from sklearn.model_selection import train_test_split
train_Xsk,test_Xsk,train_ysk,test_ysk=train_test_split(X_np,y_np)

In [56]:
train_Xsk1,train_Xsk2,train_Xsk3=np.array_split(train_Xsk,3)
train_ysk1,train_ysk2,train_ysk3=np.array_split(train_ysk,3)
test_Xsk1,test_Xsk2,test_Xsk3=np.array_split(test_Xsk,3)

In [69]:
# 第一段階目　線形回帰

from sklearn.linear_model import LinearRegression
reg21 = LinearRegression()
reg21.fit(train_Xsk1,train_ysk1)
y_pred_sk2=reg21.predict(train_Xsk2)

reg22 = LinearRegression()
reg22.fit(train_Xsk2,train_ysk2)
y_pred_sk3=reg22.predict(train_Xsk3)

reg23 = LinearRegression()
reg23.fit(train_Xsk3,train_ysk3)
y_pred_sk1=reg21.predict(train_Xsk1)

stacking_data1=np.hstack((y_pred_sk1,y_pred_sk2,y_pred_sk3))

In [67]:
# 第一段階目　線形回帰

from sklearn.linear_model import LinearRegression
reg21 = LinearRegression()
reg21.fit(train_Xsk1,train_ysk1)
y_pred_sk2=reg21.predict(train_Xsk2)

reg22 = LinearRegression()
reg22.fit(train_Xsk2,train_ysk2)
y_pred_sk3=reg22.predict(train_Xsk3)

reg23 = LinearRegression()
reg23.fit(train_Xsk3,train_ysk3)
y_pred_sk1=reg21.predict(train_Xsk1)

stacking_data1=np.hstack((y_pred_sk1,y_pred_sk2,y_pred_sk3))

In [71]:
# 第一段階　決定木

from sklearn.tree import DecisionTreeRegressor
DTR21 = DecisionTreeRegressor(random_state=0)
DTR21.fit(train_X,train_y)
y_pred2=DTR.predict(test_X)


DTR21 = DecisionTreeRegressor(random_state=0)
DTR21.fit(train_Xsk1,train_ysk1)
y_pred_DTRsk2=DTR21.predict(train_Xsk2)

DTR22 = DecisionTreeRegressor(random_state=0)
DTR22.fit(train_Xsk2,train_ysk2)
y_pred_DTRsk3=DTR22.predict(train_Xsk3)

DTR23 = DecisionTreeRegressor(random_state=0)
DTR23.fit(train_Xsk3,train_ysk3)
y_pred_DTRsk1=DTR23.predict(train_Xsk1)

stacking_data2=np.hstack((y_pred_DTRsk1,y_pred_DTRsk2,y_pred_DTRsk3))

In [75]:
# ブレンドデータの完成
blend_data=np.vstack((stacking_data1,stacking_data2)).T

In [78]:
# # 第二段階　重回帰分析

from sklearn.linear_model import LinearRegression
reg31 = LinearRegression()
reg31.fit(blend_data,train_y)
y_pred=reg31.predict(test_X)

In [83]:
#　平均二乗誤差を計算し精度を確認
from sklearn.metrics import mean_squared_error
mean_squared_error(test_y,y_pred)

6058077692.842926

単一の線形回帰、対数化をしたものとのブレンディング、５回の平均のバギングよりも、さらにこのスタッキングで平均二乗誤差が小さくなり、より精度の分析となったことがわかった。